# Docker Image Build & Deploy

In [11]:
from project_vars import project_variables

env = project_variables['environment']
project_id = project_variables['project_id']
project_name = project_variables['project_name']
region = project_variables['region']

In [2]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=E6UoAqelFsT32pzuyuuj42Zn5gbjHK&access_type=offline&code_challenge=7lu34ITjVfW0cl2IVRfrtMV3NRICDVCyb10RRzcsqbc&code_challenge_method=S256


You are now logged in as [m.jarnot@yahoo.com].
Your current project is [fs-microservices].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
!gcloud config set project {project_id}

Updated property [core/project].


In [4]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "asia.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud",
    "us.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


## Service Build & Deploy

You need to build and deploy as many docker images as you have services. You also need to update services array when services change in `project_vars.py` file. The following code block generates select field that you can use to choose what service you want to deploy.

In [6]:
import ipywidgets as widgets
from IPython.display import display

services = [
    {"name": "auth-service", "folder": "service-auth"},
    {"name": "events-service", "folder": "service-events"},
    {"name": "gateway-service", "folder": "service-gateway"},
    {"name": "storybook-service", "folder": "storybook"},
    {"name": "web-service", "folder": "web"}
]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=[(f"{s['name']} ({s['folder']})", s) for s in services],
    description='Select a service:',
    style={'description_width': 'initial'}
)

# Create output widgets to display the selected values
output_name = widgets.HTML()
output_folder = widgets.HTML()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected = change['new']
        global service_name, service_folder
        service_name = selected['name']
        service_folder = selected['folder']
        output_name.value = f"<b>service_name</b> = \"{service_name}\""
        output_folder.value = f"<b>service_folder</b> = \"{service_folder}\""

# Attach the callback to the dropdown
dropdown.observe(on_change, names='value')

# Display the widgets
display(dropdown, output_name, output_folder)

Dropdown(description='Select a service:', options=(('auth-service (service-auth)', {'name': 'auth-service', 'f…

HTML(value='')

HTML(value='')

In [19]:
# print service_name, service_folder
print(f"Building Docker Image for {service_name} in /apps/{service_folder}")

Building Docker Image for storybook-service in /apps/storybook


In [20]:
!docker build --no-cache -t {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest -f ../apps/{service_folder}/Dockerfile ../

[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (3/5)                                    docker:desktop-linux
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 486B                                          0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 744B                                       0.0s
 => [internal] load metadata for docker.io/library/node:18.20.3-alpine3.1  0.2s
 => [internal] load metadata for docker.io/library/node:18.20.3            0.2s
 => [auth] library/node:pull token for registry-1.docker.io                0.0s
[+] Building 0.4s (3/5)                                    docker:desktop-linux
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 486B                                          0.0s
 => [internal] load build definition fro

In [21]:
!docker push {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest

The push refers to repository [us-central1-docker.pkg.dev/fs-microservices/dev-repo/dev-storybook-service]

145069f8: Preparing 
b46f11d6: Preparing 
a7d078a0: Preparing 
ccf8ba19: Preparing 
ce7ef8cf: Preparing 
f1fdab11: Preparing 
145069f8: Pushed lready exists 3MBg  121.3kB/9.443MBlatest: digest: sha256:67c5c3ef3d3bf84214c9e29b8a6c0f0d357a3e6e77f0ea249e1e26e2a332f8ad size: 1786


In [22]:
!gcloud run deploy {project_id}-{env}-{service_name} --image {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest --platform managed --region={region} --allow-unauthenticated

Deploying container to Cloud Run service [fs-microservices-dev-storybook-service] in project [fs-microservices] region [us-central1]
Deploying new service...                                                       
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      
  Deploying new service...                                                     



⠛ Deploying new service...                                                     



⠹ Deploying new service...                                                     



⠼ Deploying new service...                                                     



⠶ Deploying new service...                                                     



⠧ Deploying new service...                                                     


  ⠧ Setting IAM Policy...                         